📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Residual Net](#toc2_)    
  - [Custom ResNet](#toc2_1_)    
    - [ResNet-18](#toc2_1_1_)    
    - [ResNet-34](#toc2_1_2_)    
    - [ResNet-50](#toc2_1_3_)    
    - [ResNet-101](#toc2_1_4_)    
    - [ResNet-152](#toc2_1_5_)    
  - [PyTorch ResNet](#toc2_2_)    
    - [ResNet-18](#toc2_2_1_)    
    - [ResNet-34](#toc2_2_2_)    
    - [ResNet-50](#toc2_2_3_)    
    - [ResNet-101](#toc2_2_4_)    
    - [ResNet-152](#toc2_2_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)
   - torchvision models:
      - class
         - brings in the model class directly
         - Allows more control and customization since you are dealing directly with the class. You can override methods, customize initialization, etc.
      - function
         - This import brings in a function that returns an instance of the model
         - Easier and quicker to use, especially for standard models
   - [pytorch.org/vision/stable/models.html](https://pytorch.org/vision/stable/models.html)

In [ ]:
import torch
from torch import nn
from torch.functional import F
from torchinfo import summary
from torchvision.models import ResNet, resnet18, resnet34, resnet50, resnet101, resnet152

# <a id='toc2_'></a>[Residual Net](#toc0_)
   - Residual Net (ResNet), developed in 2015 by [Kaiming He](https://scholar.google.com/citations?user=DhtAFkwAAAAJ) and collaborators from [Microsoft Research](https://www.microsoft.com/en-us/research/)
   - It is based on the [Deep Residual Learning for Image Recognition](https://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html) paper
   - It was trained on the [ImageNet](https://www.image-net.org/) dataset (first resized to 256x256 then center cropped to 224x224) [[ImageNet viewer](https://navigu.net/#imagenet)]
   - Known for its innovative use of `residual connections` (skip connections) to improve gradient flow directly through the network, mitigating the vanishing gradient problem
   - It comes in several variants, primarily `ResNet-18`, `ResNet-34`, `ResNet-50`, `ResNet-101` and `ResNet-152`, indicating the depth of the Network
   - The `winner` of the ImageNet Large Scale Visual Recognition Challenge ([ILSVRC](https://image-net.org/challenges/LSVRC/2015/)) in 2015

<figure style="text-align: center;">
    <img src="../../../assets/images/original/cnn/architectures/resnet.svg" alt="resnet-architecture.svg" style="width: 100%;">
    <figcaption>ResNet Architecture</figcaption>
</figure>

## <a id='toc2_1_'></a>[Custom ResNet](#toc0_)
   - `Softmax` is missing due to internal implementation of `LogSoftmax` in the `CrossEntropyLoss` function.

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, channels, stride=1) -> None:
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class BottleNeck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, channels, stride=1) -> None:
        super(BottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.conv3 = nn.Conv2d(channels, self.expansion * channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000) -> None:
        super(ResNet, self).__init__()

        self.in_channels = 64

        # 3x224x224 -> 64x112x112
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)

        # 64x112x112 -> 64x112x112
        self.bn1 = nn.BatchNorm2d(64)

        # 64x112x112 -> 64x56x56
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # resnet-18 or resnet-34              : 64x56x56 -> 64x56x56
        # resnet-50, resnet-101 or resnet-152 : 64x56x56 -> 256x56x56
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)

        # resnet-18 or resnet-34              : 64x56x56  -> 128x28x28
        # resnet-50, resnet-101 or resnet-152 : 256x56x56 -> 512x28x28
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)

        # resnet-18 or resnet-34              : 128x28x28 -> 256x14x14
        # resnet-50, resnet-101 or resnet-152 : 512x28x28 -> 1024x14x14
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)

        # resnet-18 or resnet-34              : 256x14x14  -> 512x7x7
        # resnet-50, resnet-101 or resnet-152 : 1024x14x14 -> 2048x7x7
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # resnet-18 or resnet-34              : 512x7x7 -> 512x1x1
        # resnet-50, resnet-101 or resnet-152 : 2048x7x7 -> 2048x1x1
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # resnet-18 or resnet-34              : 512  -> 1000
        # resnet-50, resnet-101 or resnet-152 : 2048 -> 1000
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, channels, num_blocks, stride) -> nn.Sequential:
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, channels, stride))
            self.in_channels = channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        # feature extractor
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # adaptive average pooling
        x = self.avgpool(x)

        # flatten:
        # resnet-18 or resnet-34              : 512x1x1  -> 512
        # resnet-50, resnet-101 or resnet-152 : 2048x1x1 -> 2048
        x = torch.flatten(x, start_dim=1)

        # classifier
        x = self.fc(x)

        return x

### <a id='toc2_1_1_'></a>[ResNet-18](#toc0_)

In [ ]:
resnet_18_1 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=1000)
resnet_18_1

In [ ]:
summary(resnet_18_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_2_'></a>[ResNet-34](#toc0_)

In [ ]:
resnet_34_1 = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=1000)
resnet_34_1

In [ ]:
summary(resnet_34_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_3_'></a>[ResNet-50](#toc0_)

In [ ]:
resnet_50_1 = ResNet(BottleNeck, [3, 4, 6, 3], num_classes=1000)
resnet_50_1

In [ ]:
summary(resnet_50_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_4_'></a>[ResNet-101](#toc0_)

In [ ]:
resnet_101_1 = ResNet(BottleNeck, [3, 4, 23, 3], num_classes=1000)
resnet_101_1

In [ ]:
summary(resnet_101_1, (1, 3, 224, 224), device="cpu")

### <a id='toc2_1_5_'></a>[ResNet-152](#toc0_)

In [ ]:
resnet_152_1 = ResNet(BottleNeck, [3, 8, 36, 3], num_classes=1000)
resnet_152_1

In [ ]:
summary(resnet_152_1, (1, 3, 224, 224), device="cpu")

## <a id='toc2_2_'></a>[PyTorch ResNet](#toc0_)
   - ResNet is available in PyTorch: [pytorch.org/vision/main/models/resnet.html](https://pytorch.org/vision/main/models/resnet.html)

### <a id='toc2_2_1_'></a>[ResNet-18](#toc0_)

In [ ]:
resnet_18_2 = resnet18()
resnet_18_2

In [ ]:
summary(resnet_18_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_2_'></a>[ResNet-34](#toc0_)

In [ ]:
resnet_34_2 = resnet34()
resnet_34_2

In [ ]:
summary(resnet_34_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_3_'></a>[ResNet-50](#toc0_)

In [ ]:
resnet_50_2 = resnet50()
resnet_50_2

In [ ]:
summary(resnet_50_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_4_'></a>[ResNet-101](#toc0_)

In [ ]:
resnet_101_2 = resnet101()
resnet_101_2

In [ ]:
summary(resnet_101_2, (1, 3, 224, 224), device="cpu")

### <a id='toc2_2_5_'></a>[ResNet-152](#toc0_)

In [ ]:
resnet_152_2 = resnet152()
resnet_152_2

In [ ]:
summary(resnet_152_2, (1, 3, 224, 224), device="cpu")